### downloading census data for comparison with synth pop outputs

In [1]:
%load_ext autoreload
%autoreload 2
from synthpop.census_helpers import Census
from synthpop import categorizer as cat
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns', 500)

This notebook is being used to process ACS data obtained from the ACS API. 

First some notes about the structure of how synthpop works. 
It has a few dependencies: 
- **census**, a python library that is a wrapper for the U.S. Census Bureau API
- **pandas** a python library used to create and work with dataframes (like tables)
- **numpy**
- **os** which allows you to set up a development environment (not actually entirely necessary)

SynthPop itself is actually a few separate scripts that each handle different aspects of the synthesizing process: 
The Synth pop library **census_helpers.py** relies on **census** and is a set of funcitons to assist with downloading and processing census data for a given geography. It allows you to select geography and columns of interest and to download data at the block or tract level (or both)
**zone_synthesizer.py** is a set of functions that accepts marginals and sample files from a CSV and produces a synthesized population
**synthesizer.py** uses a 'recipe' which is the output of the **starter.py** script
**starter.py** uses **census_helpers.py** to generate and return: 
- household marginals
- person marginals
- household joint distribution
- person joint distribution
- tract to PUMA map (a disctionary showing the relationship between tracts and PUMAs)


**Step 1:**  
Set API key. (If you don't already have you can get one [here](https://api.census.gov/data/key_signup.html))

In [2]:
# Dare's Census API key
c = Census("d95e144b39e17f929287714b0b8ba9768cecdc9f")

**Step 2:**  
Set up state and county variables to use for the state/county you are working with

In [3]:
# set state to North Carolina
stateFips = "37"
# set county to Mecklenburg
countyFips = "119"

**Step 3:**  
Define household variables of interest

In [28]:
income_columns = ['B19001_0%02dE'%i for i in range(1, 18)]
income_columns_moe = ['B19001_0%02dM'%i for i in range(1, 18)]
vehicle_columns = ['B08201_0%02dE'%i for i in range(1, 7)]
vehicle_columns_moe = ['B08201_0%02dM'%i for i in range(1, 7)]

workers_columns = ['B08202_0%02dE'%i for i in range(1, 6)]
families_columns = ['B11001_001E', 'B11001_002E']
families_columns_moe = ['B11001_001M', 'B11001_002M']
# traveltimesex_columns = ['B08013_001E','B08013_002E','B08013_003E']
census_col = families_columns + families_columns_moe+income_columns + income_columns_moe + vehicle_columns + vehicle_columns_moe
h_acs = c.tract_query(census_col, stateFips, countyFips)


In [6]:
##Note: it is also possible to do this at the block and tract level. See example below. 
## We are not using this
income_columns = ['B19001_0%02dE'%i for i in range(1, 18)]
vehicle_columns = ['B08201_0%02dE'%i for i in range(1, 7)]
workers_columns = ['B08202_0%02dE'%i for i in range(1, 6)]
families_columns = ['B11001_001E', 'B11001_002E']
# traveltimesex_columns = ['B08013_001E','B08013_002E','B08013_003E']
block_group_columns = income_columns + families_columns
tract_columns = vehicle_columns + workers_columns 
# + traveltimesex_columns
#function for getting block and tract level data. merges to the block level
h_acs_block = c.block_group_and_tract_query(block_group_columns,
                tract_columns, stateFips, countyFips, 
                merge_columns=['tract', 'county', 'state'],
                block_group_size_attr="B11001_001E",
                tract_size_attr="B08201_001E")

**Step 4:**  
Define person-level variables of interest

In [33]:
population = ['B01001_001E']
# margin of error = _moe in all columns
population_moe = ['B01001_001M']
sex = ['B01001_002E', 'B01001_026E']
sex_moe = ['B01001_002M', 'B01001_026M']
race = ['B02001_0%02dE'%i for i in range(1,11)]
race_moe = ['B02001_0%02dM'%i for i in range(1,11)]                                     
male_age_columns = ['B01001_0%02dE'%i for i in range(3,26)]
male_age_columns_moe = ['B01001_0%02dM'%i for i in range(3,26)]
female_age_columns = ['B01001_0%02dE'%i for i in range(27,50)]
female_age_columns_moe = ['B01001_0%02dM'%i for i in range(27,50)]

                                             
all_columns = population + sex + race + male_age_columns + female_age_columns+population_moe+sex_moe+race_moe+male_age_columns_moe+female_age_columns_moe
p_acs_tract = c.tract_query(all_columns, stateFips, countyFips)


In [24]:
##calculate standard error for person level columns
margin_error = [population_moe, sex_moe, race_moe, male_age_columns_moe,female_age_columns_moe]


0.21888412017167383

0.21888412017167383

### Calculating aggregate errors for our columns. 
Standard Error  = Margin of Error / Z

See [this](https://www2.census.gov/programs-surveys/acs/tech_docs/statistical_testing/2015StatisticalTesting5year.pdf) guide from ACS for further info: 

In [ ]:
p_acs_tract.loc[:,]

In [ ]:
for 

def standard_error(x):
    if 
if column name is in list of column names: 
    return x/1.645
else:
    return

**Step 5:**  
Categorize the ACS household data into categories we can read

In [30]:
h_acs_cat = cat.categorize(h_acs, {
    ("households", "total"): "B11001_001E",
    ("children", "yes"): "B11001_002E",
    ("children", "no"): "B11001_001E - B11001_002E",
    ("income", "lt35"): "(B19001_002E + B19001_003E + B19001_004E + "
                        "B19001_005E + B19001_006E + B19001_007E)/1.645",
    ("income", "gt35-lt100"): "B19001_008E + B19001_009E + "
                        "B19001_010E + B19001_011E + B19001_012E"
                        "+ B19001_013E",
    ("income", "gt100"): "B19001_014E + B19001_015E + B19001_016E"
                        "+ B19001_017E",
    ("cars", "none"): "B08201_002E",
    ("cars", "one"): "B08201_003E",
    ("cars", "two or more"): "B08201_004E + B08201_005E + B08201_006E",
#     ("workers", "none"): "B08202_002E",
#     ("workers", "one"): "B08202_003E",
#     ("workers", "two or more"): "B08202_004E + B08202_005E",
#     ("traveltime","total"):"B08013_001E",
#     ("traveltime","male"):"B08013_002E",
#     ("traveltime","female"):"B08013_003E"
}, index_cols=['tract'])
h_acs_cat

cat_name  cars                   children       households income             \
cat_value none   one two or more       no   yes      total  gt100 gt35-lt100   
tract                                                                          
000100     426  1327        1072     2266   559       2825   1476        867   
000300     155   206         120      410    71        481    152         93   
000400     278  1014         472     1511   253       1764    650        794   
000500     441  1271         783     1891   604       2495    944        921   
000600     235   832         502     1081   488       1569    379        644   
000700      16   137         206      182   177        359     38        244   
000800     207   412         312      380   551        931     49        306   
000900     116   292         307      424   291        715     79        269   
001000      61   365         766      613   579       1192    586        369   
001100      25   330         577      468   464        932    414        389   
001200     302   824        1080     1029  1177       2206    647        800   
001300     242   654         781      918   759       1677    141        585   
001400      40   578         649      707   560       1267    427        596   
001504     224  1155         999      922  1456       2378     60       1141   
001505       9   521         716      344   902       1246    224        638   
001507     322   587         467      504   872       1376     33        482   
001508      30   776        1427      823  1410       2233    260       1300   
001509     112   501         558      464   707       1171    109        530   
001510     168   433         622      498   725       1223     95        431   
001603     389   849         608     1016   830       1846    153        717   
001605     158   344         305      410   397        807     70        325   
001606     249   713         640      748   854       1602     88        717   
001607     258   425         499      651   531       1182     39        408   
001608     182   683         677      723   819       1542     38        695   
001609     149   621         490      610   650       1260    124        461   
001701     273   810         571      871   783       1654    191        689   
001702     340  1086         637      905  1158       2063     86        913   
001801      70   298         256      353   271        624     68        238   
001802     102   769         644      898   617       1515    242        790   
001910     214   834         735      631  1152       1783    111        736   
...        ...   ...         ...      ...   ...        ...    ...        ...   
006009       5   520         661      484   702       1186    153        681   
006010     195   644        1510      719  1630       2349    294       1184   
006103      43   427         988      401  1057       1458    610        572   
006104      52  1036        2101      875  2314       3189    614       1723   
006105      31   217         739      223   764        987    308        473   
006106      38   708        1046      883   909       1792    409        999   
006107      22  1100        1138     1105  1155       2260    513       1330   
006108      32   321        1207      464  1096       1560    295        878   
006109      80   493         544      456   661       1117    192        557   
006203      30   801        1564     1012  1383       2395   1155        649   
006204      56   860        2277      917  2276       3193   1827       1028   
006208       7   564         766      553   784       1337    445        786   
006209      65  1005        1159     1148  1081       2229    778        924   
006210      34   437        1176      445  1202       1647    878        564   
006211      31   553        1541      430  1695       2125   1221        767   
006212      10   566         823      536   863       1399    479        679   


**Step 8:**  
Categorize the ACS person data into categories we can read

In [10]:
p_acs_cat = cat.categorize(p_acs_tract, {
    ("population", "total"): "B01001_001E",
    ("age", "19 and under"): "B01001_003E + B01001_004E + B01001_005E + "
                             "B01001_006E + B01001_007E + B01001_027E + "
                             "B01001_028E + B01001_029E + B01001_030E + "
                             "B01001_031E",
    ("age", "20 to 35"): "B01001_008E + B01001_009E + B01001_010E + "
                         "B01001_011E + B01001_012E + B01001_032E + "
                         "B01001_033E + B01001_034E + B01001_035E + "
                         "B01001_036E",
    ("age", "35 to 60"): "B01001_013E + B01001_014E + B01001_015E + "
                         "B01001_016E + B01001_017E + B01001_037E + "
                         "B01001_038E + B01001_039E + B01001_040E + "
                         "B01001_041E",
    ("age", "above 60"): "B01001_018E + B01001_019E + B01001_020E + "
                         "B01001_021E + B01001_022E + B01001_023E + "
                         "B01001_024E + B01001_025E + B01001_042E + "
                         "B01001_043E + B01001_044E + B01001_045E + "
                         "B01001_046E + B01001_047E + B01001_048E + "
                         "B01001_049E", 
    ("race", "white"):   "B02001_002E",
    ("race", "black"):   "B02001_003E",
    ("race", "asian"):   "B02001_005E",
    ("race", "other"):   "B02001_004E + B02001_006E + B02001_007E + "
                         "B02001_008E",
    ("sex", "male"):     "B01001_002E",
    ("sex", "female"):   "B01001_026E"
}, index_cols=['tract'])
p_acs_cat

cat_name           age                            population  race        \
cat_value 19 and under 20 to 35 35 to 60 above 60      total asian black   
tract                                                                      
000100             244     2713     1544      430       4931   289   501   
000300              18      204      340       83        645    24   150   
000400              52     1751      661      176       2640   113   407   
000500             834     2791      981      376       4982   149  1278   
000600             409     1588      671      204       2872     9  1140   
000700             120      262      344       87        813    12   406   
000800            1008      822      849      213       2892    16  1969   
000900             322      516      638      292       1768    33  1273   
001000             463      901      942      257       2563    46   207   
001100             437      685      983      164       2269    18   168   
001200            1088     1172     2245      757       5262   485  1081   
001300             661     1013     1359      746       3779   163  1542   
001400             368      945     1068      190       2571   112   562   
001504            2841     1418     2265      937       7461    41  4306   
001505            1046      754     1229      534       3563   103  1353   
001507            1297     1133     1222      517       4169     0  2450   
001508            1773     1306     2170      959       6208   136  2697   
001509             837      698     1022      660       3217    62  2201   
001510             692      484     1110      649       2935   193  2118   
001603             939     1105     1204     1176       4424   342  1296   
001605             417      588      689      217       1911   144   508   
001606            1353      881     1629      626       4489   503  1212   
001607             781      796      939      320       2836    83  1165   
001608            1078     1009     1344      537       3968    62  1635   
001609             830      918     1046      340       3134   214  1411   
001701            1091     1653     1173      360       4277   678  1243   
001702            1460     1786     1608      651       5505   747  1329   
001801             256      441      554       96       1347    13   323   
001802             491     1051     1037      445       3024   328   950   
001910            1894     1692     1664      552       5802   406  2393   
...                ...      ...      ...      ...        ...   ...   ...   
006009             728      699     1051      369       2847   132  1403   
006010            2555     1228     2174      834       6791   130  3548   
006103            1051      608     1660      485       3804   113   876   
006104            2670     2355     3442      995       9462   242  4995   
006105             930      560     1201      319       3010   113  1309   
006106             822     1170     1620      458       4070   146  1582   
006107            1282     1747     1918      331       5278   156  2696   
006108            1447     1135     1658      465       4705   101  3104   
006109             962     1571     1472      604       4609     0  3747   
006203             929      738     2304     1085       5056    12   165   
006204            1613      510     3176     2027       7326   160   249   
006208             809      907      947      382       3045   123   203   
006209             863     1076     1902      401       4242    85   366   
006210            1254      705     1700      734       4393   265   195   
006211            2168      679     2676      699       6222    72   418   
006212             866      972     1265      384       3487   176   439   
006213            1912     1132     2379      613       6036   116   591   
006214            1173      724     1614      620       4131    28   481   
006215            1197     1056     1757      54

In [13]:
p_acs_cat.to_csv('data_outputs/20190326_census_aggregates/37119_people_meck.csv')
h_acs_cat.to_csv('data_outputs/20190326_census_aggregates/37119_households_meck.csv')